# Live Tutorial 1b - Single-particle tracking in Python - Part I

----------
## Qbio Summer School 2021
--------------

```
Instructor: Luis U. Aguilera
Author: Luis U. Aguilera
Contact Info: luis.aguilera@colostate.edu

Copyright (c) 2021 Dr. Brian Munsky. 
Dr. Luis Aguilera, Will Raymond
Colorado State University.
Licensed under MIT License.
```


<img src= https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/presentation/Module_1b/Slide1.png alt="drawing" width="1200"/>

# Abstract 

This notebook provides a implement a single particule simulator and tracker. At the end of the tutorial, the student is expected to acquire the computational skills to implement the following list of objectives independently.

## List of objectives

1.   To simulate 2D random walks.
2.   To implement a code to track single particles.
3.   To calculate mean square displacement and diffusion coefficients.

In [1]:
# Loading libraries
import matplotlib.pyplot as plt # Library used for plotting
from matplotlib import animation # Library to plot animations
import numpy as np # library for array manipulation
from random import randrange # Library to generate random numbers
import math # Library for math calculations
import skimage # Library for image manipulation
from skimage.util import random_noise # module to add random noise to the image
from scipy.spatial import KDTree # module to link trajectories
from skimage import measure # Module to find contours on images
from IPython.display import HTML # To display a mp4 video

<img src= https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/presentation/Module_1b/Slide3.png alt="drawing" width="1200"/>

# Particle Simulator

<img src= https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/presentation/Module_1b/Slide5.png alt="drawing" width="1200"/>

### Code for a 2-D random walk. To access it, double-click in the cell below.

In [2]:
#@title
def brownian_movement_simulator(img_size = [100,100] ,num_time_points=10, number_spots = 20, diffusion_coeffient=1,percentage_background_noise=0):
  '''
  This function is intended to create spots and simulated brownian motion.
  Inputs:
    img_size: list of two int, with format [x_size, y_size]. Units are pixels.
    num_time_points: int, number of time points to simulate
    num_spots: int, number of spots to simulate.
    diffusion_coeffient: float.

  Returns
    img_with_spots: numpy array with dimenssions [T,Y,X,C]
  '''
  #####
  ##### Step 1. Generating an empty array
  #####
  img = np.zeros((num_time_points, img_size[0], img_size[1], 1),dtype=np.uint8) 
  ## Function parameters.
  initial_points = np.zeros((number_spots,2 ),dtype=np.uint16) 
  size_x = img.shape[2]
  size_y = img.shape[1]
  step_size = 1
  num_time_points = img.shape[0]
  min_space_to_avoid_edges = 5 # minimal number of pixeles closer to a border 
  size_spot = 5
  spot_sigma = 2
  #####
  ##### Step 2. Replacing ""Particles"" as a 2-D Gaussian matrix
  #####
  ax = np.linspace(-(size_spot - 1) / 2., (size_spot - 1) / 2., size_spot)
  xx, yy = np.meshgrid(ax, ax)
  kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(spot_sigma)) 
  kernel = (kernel / np.amax(kernel)) * 255

  #####
  ##### Step 3. Generating initial positions
  #####
  for i in range (0,number_spots):
    initial_points[i,:] = [randrange(min_space_to_avoid_edges,size_x-min_space_to_avoid_edges), randrange(min_space_to_avoid_edges,size_y-min_space_to_avoid_edges)]
  ## Brownian motion  
  brownian_movement = math.sqrt(2*diffusion_coeffient*step_size) # Scaling factor for Brownian motion.
  # Prealocating memory
  y_positions = np.array(initial_points[:,0],dtype='int') #  x_position for selected spots inside the image
  x_positions = np.array(initial_points[:,1],dtype='int') #  y_position for selected spots inside the image
  spot_positions_movement = np.zeros((num_time_points,number_spots,2),dtype='int')
  # Temporal array with positions
  temp_Position_y = np.zeros_like(y_positions,dtype='int')
  temp_Position_x = np.zeros_like(x_positions,dtype='int')
  # Updated array with positions
  newPosition_y = np.zeros_like(y_positions,dtype='int')
  newPosition_x = np.zeros_like(x_positions,dtype='int')
  
  #####
  ##### Step 4. Main loop that computes the random motion and new spot positions
  #####
  for t_p in range(0,num_time_points):
      for i_p in range (0, number_spots):
          if t_p == 0:
              temp_Position_y[i_p]= y_positions[i_p]
              temp_Position_x[i_p]= x_positions[i_p]            
          else:
              temp_Position_y[i_p]= newPosition_y[i_p] + int(brownian_movement * np.random.randn(1))
              temp_Position_x[i_p]= newPosition_x[i_p] + int(brownian_movement * np.random.randn(1))
          # Test that spots are not going outside the image.
          if temp_Position_y[i_p] < min_space_to_avoid_edges or temp_Position_y[i_p] > size_y-min_space_to_avoid_edges or temp_Position_x[i_p] < min_space_to_avoid_edges or temp_Position_x[i_p] > size_x-min_space_to_avoid_edges :
              temp_Position_y[i_p]= newPosition_y[i_p] 
              temp_Position_x[i_p]= newPosition_x[i_p] 
          # Updating positions
          newPosition_y[i_p]= temp_Position_y[i_p]
          newPosition_x[i_p]= temp_Position_x[i_p]
      # Final numpy array with all spots for all time points
      spot_positions_movement [t_p,:,:]= np.vstack((newPosition_y, newPosition_x)).T
  # Replacing pixels where a spot should be located with a Gaussian matrix
  img_with_spots = img.copy()
  for t_p in range(0,num_time_points):
    for i_p in range (0, number_spots):
      center_position = spot_positions_movement[t_p,i_p,:]
      img_with_spots[t_p, center_position[0]-round(size_spot/2): center_position[0]+round(size_spot/2)+1 ,center_position[1]-round(size_spot/2): center_position[1]+round(size_spot/2)+1,0 ] = kernel
  # Adding background noise
  if percentage_background_noise != 0:  
    img_with_spots= random_noise(img_with_spots, mode='gaussian', mean=percentage_background_noise, var=percentage_background_noise/2) # converts to float
    img_with_spots = np.array(255 * img_with_spots, dtype=np.uint8) # converting back to 8-bit
  # Retuning a numpy array with the simulated data
  return img_with_spots

## Running the simulator

In [3]:
# Running the function
img_size =[100, 100]
num_time_points = 50
diffusion_coeffient = 0.5
number_spots = 10
img_with_spots = brownian_movement_simulator(img_size =img_size,num_time_points=num_time_points, number_spots=number_spots, diffusion_coeffient =diffusion_coeffient, percentage_background_noise= 0)

### Plotting results as a video

In [ ]:
#@title
# Plotting spots as a video
fig = plt.figure(figsize=(7,7))
#Define inital frames
im = plt.imshow(img_with_spots[0,:,:,0],cmap= 'Reds_r') # Reds_r
#plt.axis('off')
def movieFrame(i):
  images = [img_with_spots[i,:,:,0]]
  image_handles = [im]
  for k, image_n in enumerate(images):
    image_handles[k].set_array(images[k])
  return image_handles
plt.close()
anim = animation.FuncAnimation(fig, movieFrame, frames=img_with_spots.shape[0], interval=50, blit=True)
from IPython.display import HTML
HTML(anim.to_html5_video())

In [ ]:
# 3D-Visualization (X,Y, intensity)
fig = plt.figure(dpi=120)
ax1 = np.arange(0,img_with_spots.shape[1],1)
xx, yy = np.meshgrid(ax1, ax1)
ax_3D = plt.axes(projection='3d')
ax_3D.plot_surface(xx, yy, img_with_spots[0,:,:,0], cmap='Reds')
plt.show()

# Particle Tracking Process

<img src= https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/presentation/Module_1b/Slide6.png alt="drawing" width="1200"/>

## Particle detection

### Binarization

In [6]:
treshold = 100  # Values in range [0, 255]

In [7]:
# Define an intensity treshold
selected_time_point = 1
img_tracking = img_with_spots.copy() # copy of the image.
slected_tp_img = img_tracking[selected_time_point,:,:,0] # selecting a time point

## Image binarization
# Making spots above the treshold equal to 255. The maximum value in a 8-bit image.
slected_tp_img[slected_tp_img>treshold] = 255
# Making spots below the treshold equal to 0.
slected_tp_img[slected_tp_img<treshold] = 0
# Binarization
slected_tp_img[slected_tp_img!=0] = 1

In [ ]:
# Plotting 
fig, ax = plt.subplots(1,2, figsize=(15, 5))
ax[0].imshow(img_with_spots[selected_time_point,:,:,0],cmap='Greens_r')
ax[0].set_title('Simulated spots')
ax[1].imshow(slected_tp_img,cmap='Greys_r')
ax[1].set_title('Binary image with pixels above treshold')
plt.show()

In [ ]:
# Joining pixels in "particles"
contours = measure.find_contours(slected_tp_img, 0.5)
# Display the image and plot all contours found
fig, ax = plt.subplots()
ax.imshow(slected_tp_img, cmap=plt.cm.gray)
for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

### Detecting particles (center of mass) for all frames

In [10]:
def particle_detector(img_with_spots,threshold):
  '''
  '''
  img_tracking = img_with_spots.copy() # copy of the image.
  num_time_points = img_tracking.shape[0]
  list_all_time_point_center_mass  =[]
  for i in range (0,num_time_points):
    list_center_mass = []
    slected_tp_img = img_tracking[i,:,:,0] # selecting a time point
    # Making spots above the threshold equal to 255.
    slected_tp_img[slected_tp_img>threshold] = 255
    # Making spots below the threshold equal to 0.
    slected_tp_img[slected_tp_img<threshold] = 0
    # Binarization
    slected_tp_img[slected_tp_img!=0] = 1
    # Joining pixels and deffining particles
    contours = measure.find_contours(slected_tp_img, 0.5)
    # Calculating the center of mass of each particle
    for contour in contours:
      center_mass = [np.median(contour[:, 1]), np.median(contour[:, 0]) ]  # y and x
      list_center_mass.append(center_mass)
    # Saving results as a list of centers of mass for each time point.  [ [ cm_particle_0_tp_0, ..., cm_particle_n_tp_0], ... , [ cm_particle_0_tp_n, ..., cm_particle_n_tp_n] ]
    list_all_time_point_center_mass.append(list_center_mass)
  return list_all_time_point_center_mass 

In [11]:
list_all_time_point_center_mass = particle_detector(img_with_spots,threshold=100)

In [12]:
# List containing [Y,X] positions for the center of mass for each particle. 
list_all_time_point_center_mass[0] # Selecting a time point

[[46.0, 9.0],
 [19.0, 28.0],
 [54.0, 44.0],
 [43.0, 55.0],
 [12.0, 58.0],
 [77.0, 66.0],
 [28.0, 75.0],
 [82.0, 81.0],
 [24.0, 93.0]]

In [ ]:
# Plotting center of mass of each particle
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(img_with_spots[selected_time_point,:,:,0], cmap=plt.cm.gray)
for i in range(0,len(list_all_time_point_center_mass[selected_time_point])):
  ax.plot(list_all_time_point_center_mass[selected_time_point][i][0], list_all_time_point_center_mass[selected_time_point][i][1], color='r',marker='*')
plt.show()

## Linking trajectories

In [14]:
def linking_spots(list_all_time_point_center_mass):
  '''
  '''
  def get_points_on_slice(i):
      return np.array(list_all_time_point_center_mass[i])
  get_points_on_slice
  # Look for the nearest point slice by slice:
  n_last_slice = len (list_all_time_point_center_mass)-1
  start_points = np.array(list_all_time_point_center_mass[0])
  idx_nearest_trajectories = np.arange(start_points.shape[0]).reshape(1, -1)

  # Loop for all frames
  for i in range(1, n_last_slice+1):
      get_nearest = KDTree(get_points_on_slice(i))
      previous_points = get_points_on_slice(i-1)[idx_nearest_trajectories[-1, :]]
      distance, idx_nearest = get_nearest.query(previous_points)
      idx_nearest_trajectories = np.vstack((idx_nearest_trajectories, idx_nearest))
  list_trajectories =[]
  for path_idx in idx_nearest_trajectories.T:
    path_coords = [list_all_time_point_center_mass[i][idx] for i, idx in enumerate(path_idx)]
    list_trajectories.append(path_coords)
  return list_trajectories

In [15]:
list_trajectories_short = linking_spots (list_all_time_point_center_mass)

In [ ]:
# Plotting individual trajectories
plt.figure(figsize=(6, 6))
for i in range(0, len(list_trajectories_short)):
    plt.plot(*zip(*list_trajectories_short[i]), '-', linewidth=2);
plt.xlim([0, img_size[1]])
plt.ylim([img_size[0], 0])
plt.show()

# Calculating the Mean Square Displacement

<img src= https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/presentation/Module_1b/Slide7.png alt="drawing" width="1200"/>

In [17]:
def compute_MSD(path):
  '''
  '''
  totalsize=len(path)
  msd=[]
  for i in range(totalsize-1):
      j=i+1
      msd.append(np.sum((path[0:-j]-path[j::])**2)/float(totalsize-j)) # Distance that a particle moves for each time point divided by time
  msd=np.array(msd)
  rmsd = np.sqrt(msd)
  return msd, rmsd 

In [18]:
# Running the function
img_size_kd =[2000, 2000]
num_time_points = 100
diffusion_coeffient = 0.5
percentage_background_noise = 0
number_spots = 100
img_with_spots_kd = brownian_movement_simulator(img_size =img_size_kd,num_time_points=num_time_points, number_spots=number_spots, diffusion_coeffient =diffusion_coeffient, percentage_background_noise= percentage_background_noise)

In [19]:
# Detecting particles
list_center_mass_kd = particle_detector(img_with_spots_kd,threshold=200)

In [20]:
# Linking trajectories
list_trajectories_kd = linking_spots (list_center_mass_kd)

In [21]:
num_trajectories = len(list_trajectories_kd)

In [22]:
# Calculating the MSD
t=np.arange(1,num_time_points,1)
msd_trajectories = np.zeros((num_trajectories,num_time_points-1))
rmsd_trajectories = np.zeros((num_trajectories,num_time_points-1))
for i in range(0,num_trajectories):
  msd_trajectories[i,:], rmsd_trajectories[i,:] = compute_MSD(np.array(list_trajectories_kd[i]))

In [23]:
# MSD Statistics (mu, sigma) for all trajectories.
msd_trajectories_all_mu = np.mean(msd_trajectories,axis=0)
msd_trajectories_all_sem = np.std(msd_trajectories,axis=0) /np.sqrt(num_trajectories)

In [ ]:
# Plotting the MSD vs Time
down_sampling = 5
plt.figure(figsize=(7, 5))
plt.errorbar(t[::down_sampling], msd_trajectories_all_mu[::down_sampling],  yerr=msd_trajectories_all_sem[::down_sampling],ecolor='grey',linestyle='')
plt.plot(t[::down_sampling], msd_trajectories_all_mu[::down_sampling], marker='o', markersize=12, linestyle='none',color='grey',label ='simulation' )
plt.plot(t, 2*diffusion_coeffient*t, color='red', linewidth=4,label ='msd_formula')
plt.legend()
plt.title('Mean square displacement')
plt.ylabel('MSD (au)')
plt.xlabel('Time (au)')
plt.show()

## Estimating the Diffusion Coefficient.

In [25]:
calculated_k_diff = msd_trajectories_all_mu / (2*t)

In [ ]:
plt.figure(figsize=(7, 5))
plt.errorbar(t[::down_sampling], calculated_k_diff[::down_sampling],  yerr=msd_trajectories_all_sem[::down_sampling],ecolor='grey',linestyle='')
plt.plot(t[::down_sampling], calculated_k_diff[::down_sampling], marker='o', markersize=12, linestyle='none',color='grey',label ='k_diff_simulation' )
plt.plot( t, np.ones((t.shape))*diffusion_coeffient, color='red', linewidth=4,label ='real_k_diff = '+str(diffusion_coeffient))
plt.legend()
plt.title('Diffusion coefficient')
plt.ylabel('k_diff (au)')
plt.xlabel('Time (au)')
plt.ylim([0.1, 1])
plt.show()

# More complex scenarios

<img src= https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/presentation/Module_1b/Slide8.png alt="drawing" width="1200"/>

## Running a simulation with 20% of Background noise.

In [27]:
# Running the function
img_size_ns =[100, 100]
num_time_points = 10
diffusion_coeffient = 0.5
percentage_background_noise = 0.2   ### adding 20% BG noise
number_spots = 30
img_with_spots_noisy = brownian_movement_simulator(img_size =img_size_ns,num_time_points=num_time_points, number_spots=number_spots, diffusion_coeffient =diffusion_coeffient, percentage_background_noise= percentage_background_noise)

In [28]:
list_center_mass = particle_detector(img_with_spots_noisy,threshold=200)  # Test different values for the threshold

In [ ]:
# Plotting 
selected_time_point =0
fig, ax = plt.subplots(1,2, figsize=(15, 5))
ax[0].imshow(img_with_spots_noisy[selected_time_point,:,:,0], cmap=plt.cm.gray, alpha=0.5)
ax[0].set_title('Simulated spots')
ax[1].imshow(img_with_spots_noisy[selected_time_point,:,:,0], cmap=plt.cm.gray, alpha=0.5)
for i in range(0,len(list_center_mass[selected_time_point])):
  ax[1].plot(list_center_mass[selected_time_point][i][0], list_center_mass[selected_time_point][i][1], color='r',marker='*')
ax[1].set_title('Binary image with pixels above treshold')
plt.show()

### Reducing the noise in the image by using filters. 

In [30]:
# Using a gaussian filter to remove noise
from scipy.ndimage import gaussian_filter
img_with_spots_gaussian_filter = img_with_spots_noisy.copy() # making a copy of our img
# Applying the filter
img_with_spots_filter = gaussian_filter(img_with_spots_gaussian_filter, sigma=2)

In [31]:
list_center_mass_removed_noise = particle_detector(img_with_spots_filter,threshold=100)

In [ ]:
# Plotting 
selected_time_point =0
fig, ax = plt.subplots(1,2, figsize=(15, 5))
ax[0].imshow(img_with_spots_filter[selected_time_point,:,:,0], cmap=plt.cm.gray)
ax[0].set_title('Simulated spots')
ax[1].imshow(img_with_spots_filter[selected_time_point,:,:,0], cmap=plt.cm.gray)
for i in range(0,len(list_center_mass_removed_noise[selected_time_point])):
  ax[1].plot(list_center_mass_removed_noise[selected_time_point][i][0], list_center_mass_removed_noise[selected_time_point][i][1], color='r',marker='*')
ax[1].set_title('Binary image with pixels above treshold')
plt.show()

# Example: Ribosomal frameshifting. Two mRNAs interact for some seconds.

In [ ]:
HTML("""<video width="560" alt="test" controls> <source src="https://github.com/MunskyGroup/uqbio2021/raw/main/module_1/tracking/bursting.mp4" type="video/mp4"></video>""")

# References

* Video source: Lyon, K., Aguilera, L.U., Morisaki, T., Munsky, B. and Stasevich, T.J., 2019. Live-cell single RNA imaging reveals bursts of translational frameshifting. Molecular cell, 75(1), pp.172-183. https://www.sciencedirect.com/science/article/pii/S1097276519303557

* Image source: MacKintosh, F.C., 2012. Active diffusion: the erratic dance of chromosomal loci. Proceedings of the National Academy of Sciences, 109(19), pp.7138-7139. https://www.pnas.org/content/109/19/7138

*   Code to generate a gaussian kernel matrix
https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy

*   Code to link coordenates
https://stackoverflow.com/questions/52129486/python-find-the-nearest-neighbor-pairs-in-a-list-of-point-coordinates

*   Code to compute mean square displacement
https://stackoverflow.com/questions/26472653/computing-the-mean-square-displacement-of-a-2d-random-walk-in-python


*   u-track (multiple-particle tracking) 
https://www.utsouthwestern.edu/labs/jaqaman/software/